In [15]:
import os
import torch
import h5py
from natsort import natsorted
from tqdm import tqdm
# 设置路径
pt_folder = '/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc12m'  # .pt 文件所在的文件夹路径
output_hdf5_file = '/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc12m_longSV_captions.h5'  # 输出的 HDF5 文件名

pt_folder = '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/dreamclipcc12m'  # .pt 文件所在的文件夹路径
output_hdf5_file = '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/dreamclipcc12m_images.h5'  # 输出的 HDF5 文件名

# 获取所有 .pt 文件名，并按自然数顺序排序
pt_files = natsorted([f for f in os.listdir(pt_folder) if f.endswith('.pt')])

# 获取张量的维度信息（假设所有张量的维度一致）
first_tensor = torch.load(os.path.join(pt_folder, pt_files[0]),weights_only=True)
dimension = first_tensor.shape[1]  # 获取每个样本的维度

# 计算总样本数
total_samples = sum([torch.load(os.path.join(pt_folder, f),weights_only=True).shape[0] for f in pt_files])

# 创建 HDF5 文件并创建数据集
with h5py.File(output_hdf5_file, 'w') as hdf5_file:
    # 创建一个大的数据集来存储所有张量
    dataset = hdf5_file.create_dataset('data', (total_samples, dimension), dtype='float32')
    
    # 将 .pt 文件的数据合并写入 HDF5
    current_index = 0
    for pt_file in tqdm(pt_files):
        file_path = os.path.join(pt_folder, pt_file)
        data = torch.load(file_path,weights_only=True)   # 加载张量
        
        num_samples = data.shape[0]  # 当前 .pt 文件中的样本数
        dataset[current_index:current_index + num_samples] = data.numpy()
        
        current_index += num_samples

print(f"Data successfully saved to {output_hdf5_file}")


/tmp/ipykernel_1526393/163818726.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  first_tensor = torch.load(os.path.join(pt_folder, pt_files[0]))
/tmp/ipykernel_1526393/

Data successfully saved to /home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/dreamclipcc12m_images.h5


In [5]:
x = h5py.File("/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_longSV_captions.h5", 'r')

In [7]:
import torch
data = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_longSV/0.pt',weights_only=True) 
data.shape

torch.Size([1024, 1024])

## Merge emebdding from two directory into one directory by averaging

In [4]:
import torch
import os
import tqdm
dir1 = '/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc3m'
dir2 = '/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_longSV'
dir_output = '/home/mila/l/le.zhang/scratch/light_align/data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_longSVshort_mixed'
os.makedirs(dir_output,exist_ok=True)
for file in tqdm(os.listdir(dir1)):
    data1 = torch.load(os.path.join(dir1,file),weights_only=True)
    data2 = torch.load(os.path.join(dir2,file),weights_only=True)
    # take average
    data = (data1 + data2) / 2
    torch.save(data,os.path.join(dir_output,file))